In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas_datareader import data as pdr
from datetime import datetime, timedelta
from scipy.optimize import minimize 

In [ ]:
START_DATE = "2019-01-01"
END_DATE = datetime.today().strftime("%Y-%m-%d")
#END_DATE = "2020-11-10"

In [ ]:
def stock_price():
    #Creates the dataset containing all stock prices and S&P 500 price "SPY"
    ticker_list = pd.read_csv("invest_list.csv")["0"].to_list() + ['SPY']
    
    # Get all Adjusted Close prices for all the tickers in our list,
    # between START_DATE and END_DATE
    all_data = pdr.get_data_yahoo(ticker_list, start=START_DATE, end=END_DATE)
    stock_raw_data = all_data["Adj Close"]
    
    #Remove rows with null price data
    stock_raw_data.dropna(how="any", axis=0, inplace=True)
    
    # Adding weekday number 
    weekday = []
    for day in stock_raw_data.index:
        weekday.append(day.weekday())
    stock_raw_data["weekday"] = weekday
    
    return stock_raw_data

In [ ]:
stock_raw_data = stock_price()

In [ ]:
stock_raw_data.tail()
#stock_new

In [ ]:
#Weight Optimization
def get_ret_vol_sr(weights):
    weights = np.array(weights)
    ret = np.sum(portfolio.mean()*weights)
    vol = np.sqrt(np.dot(weights.T,np.dot(portfolio.cov(),weights)))
    sr = ret/vol
    return np.array([ret,vol,sr])
#minimize negative sharpe ratio
def neg_sharpe(weights):
    return get_ret_vol_sr(weights)[2] * -1
# add constriant:check allocation sums to 1
def check_sum(weights):
    return np.sum(weights) - 1

In [ ]:
def Optimization():
# create constraint variable: an equition type of constrant. fun: pass in the function: check_sum
    cons = ({'type':'eq','fun':check_sum})
# create weight boundaries
    bounds = ((0,1),(0,1),(0,1),(0,1),(0,1))
# initial guess to start with
    init_guess = [0.2, 0.2, 0.2, 0.2, 0.2]
    opt_results = minimize(neg_sharpe, init_guess, method='SLSQP', bounds=bounds, constraints=cons)
    return opt_results

In [ ]:
def get_holding(total, weights):
    s1_w, s2_w, s3_w, s4_w, s5_w = weights
            
    s1_h = (total * s1_w) / BE_Price
    s2_h = (total * s2_w) / TWLO_Price
    s3_h = (total * s3_w) / ZM_Price
    s4_h = (total * s4_w) / SLP_Price
    s5_h = (total * s5_w) / DKNG_Price
    
    return s1_h.round(0), s2_h.round(0), s3_h.round(0), s4_h.round(0), s5_h.round(0)

In [ ]:
#Main
stock_new = stock_raw_data.copy()

initial_value = 10000

BE_Weight = []
TWLO_Weight = []
ZM_Weight = []
SLP_Weight = []
DKNG_Weight = []

BE_Holding = []
TWLO_Holding = []
ZM_Holding = []
SLP_Holding = []
DKNG_Holding = []

Portfolio = []
SPY_Bench = []
SPY_Holding = initial_value/stock_new["SPY"][0]
SPY_Holding = SPY_Holding.round(0)

for i in range(stock_new.shape[0]):
#for i in range(20):
    
    BE_Price = stock_new["BE"][i]
    TWLO_Price = stock_new["TWLO"][i]
    ZM_Price = stock_new["ZM"][i]
    SLP_Price = stock_new["SLP"][i]
    DKNG_Price = stock_new["DKNG"][i]
    SPY_Price = stock_new["SPY"][i]
    
    if i == 0:
        #Initial Value
        total = initial_value
    else:
        #Calculate Protofilo Value
        total = BE_Price*s1_h + TWLO_Price*s2_h + ZM_Price*s3_h + SLP_Price*s4_h + DKNG_Price*s5_h
    Portfolio.append(total)
    
    SPY_Bench.append(SPY_Holding*SPY_Price)
    
    #Exeaute Strategy
    # weekday[i] == 1, 1 is for Tuesday
    #if stock_new.weekday[i] == 1 or stock_new.weekday[i] == 3 or i == 0:
    if stock_new.weekday[i] == 1 or i == 0:
        reg_return = stock_new.iloc[i-20:i,:5]
        portfolio = np.log(reg_return/reg_return.shift(1))
        opt_results = Optimization()
        s1_h, s2_h, s3_h, s4_h, s5_h = get_holding(total, opt_results.x)
    
    BE_Weight.append(opt_results.x[0].round(2))
    TWLO_Weight.append(opt_results.x[1].round(2))
    ZM_Weight.append(opt_results.x[2].round(2))
    SLP_Weight.append(opt_results.x[3].round(2))
    DKNG_Weight.append(opt_results.x[4].round(2))

    BE_Holding.append(s1_h)
    TWLO_Holding.append(s2_h)
    ZM_Holding.append(s3_h)
    SLP_Holding.append(s4_h)
    DKNG_Holding.append(s5_h)

stock_new["BE_Weight"] = BE_Weight
stock_new["TWLO_Weight"] = TWLO_Weight
stock_new["ZM_Weight"] = ZM_Weight
stock_new["SLP_Weight"] = SLP_Weight
stock_new["DKNG_Weight"] = DKNG_Weight

stock_new["BE_Holding"] = BE_Holding
stock_new["TWLO_Holding"] = TWLO_Holding
stock_new["ZM_Holding"] = ZM_Holding
stock_new["SLP_Holding"] = SLP_Holding
stock_new["DKNG_Holding"] = DKNG_Holding

stock_new["Portfolio"] = Portfolio
stock_new["SPY_Bench"] = SPY_Bench


In [ ]:
stock_new.tail(20)
#stock_new.head(40)

In [ ]:
%matplotlib inline
#stock_new.plot()
stock_new[["BE_Weight","TWLO_Weight", "ZM_Weight", "SLP_Weight", "DKNG_Weight"]].plot(figsize=(20, 6))
stock_new[["BE_Holding","TWLO_Holding", "ZM_Holding", "SLP_Holding", "DKNG_Holding"]].plot(figsize=(20, 6))
stock_new[["Portfolio", "SPY_Bench"]].plot(figsize=(20, 6))
stock_new[["BE","TWLO", "ZM", "SLP", "DKNG"]].plot(figsize=(20, 6))
np.log(stock_new[["BE","TWLO", "ZM", "SLP", "DKNG"]]).plot(figsize=(20, 6))

# Viewing The Trend
#---------------------------------------------------------------------------------------------------------------------------------------------------
#Configuring the figure size to 15 x 7
fig, ax1 = plt.subplots(figsize=(15,7))
#Creating space to plot in the in the Sub Grid to accomadate the other plot
ax1 = plt.subplot(2,1,1)
ax1.grid(True) #Enabling Grid in the plot
plt.xlabel('Years') #Naming the X-Axis
plt.ylabel('Price ($)') #Naming the Y-Axis
plt.title('Alphabet (GOOG) Stock Price') #Naming the Title
ax1.plot(selected_data.date.values,selected_data.adj_close.values,'#10e8b2',linewidth=2) #Plotting adj_close against dates and giving the desired color
ax2 = plt.subplot(2,1,2)
ax2.bar(selected_data.date.values,selected_data.volume.values,color='#3e83c9')
ax2.grid(True)
plt.xlabel('Years') #Naming the X-Axis for plot2
plt.ylabel('Volume') #Naming the Y-Axis for plot2
plt.show()
#---------------------------------------------------------------------------------------------------------------------------------------------------

#---------------------------------------------------------------------------------------------------------------------------------------------------
##Variables to get the Raw Data from the API and Store in the File
companies = {"Alphabet":"GOOG","Microsoft":"MSFT","Apple":"AAPL"} #Dictionary of companies created for better understanding of code
store_data = get_raw_data(companies) #Calling the function and storing the value(list) 
with open('Raw_data.csv','w',newline='')as rawfile: #Creating a CSV File 
            w = csv.writer(rawfile)
            for rows in store_data: #Iterating over the retrieved List of CSV data
                w.writerow(rows) #Writing each row in the Raw_data.csv file created
            rawfile.close()
print('Raw File Saved')
#---------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#Calculate holding based on real Portfolio
current_total = 14400
get_holding(current_total, opt_results.x)

In [ ]:
#Caculate hoding difference
208-199

In [ ]:
stock_new.to_csv("Portfolio_Backtest_Result.csv")

In [ ]:
#Portflio performance
max_cash=stock_new["Portfolio"].max()
min_cash=stock_new["Portfolio"].min()
final_cash=stock_new["Portfolio"].tail(1)

In [ ]:
print("Initial Cash: $",10000)
print("Max Cash: $", max_cash)
print("Min Casg: $",min_cash)
print("Final Cash: $",final_cash)

In [ ]:
#Portflio evaluation
#https://www.investopedia.com/articles/07/sharpe_ratio.asp#:~:text=Using%20the%20Sharpe%20Ratio,A%20is%20a%20better%20performer.
#https://www.investopedia.com/terms/m/maximum-drawdown-mdd.asp#:~:text=A%20maximum%20drawdown%20(MDD)%20is,over%20a%20specified%20time%20period.
daily_return = np.log(stock_new["Portfolio"]/stock_new["Portfolio"].shift(1))
#daily_return = stock_new["Portfolio"]-stock_new["Portfolio"].shift(1)
ret = daily_return.mean()
vol = daily_return.std()
sr = ret/vol

def MaxDrawdown(return_list):
    a = np.maximum.accumulate(return_list)
    l = np.argmax((a - return_list) /a)
    k = np.argmax(return_list[:l])
    return (return_list[k] - return_list[l])/(return_list[k])

return_list = list(stock_new["Portfolio"])
mdd = MaxDrawdown(return_list)
drawdown = round(mdd*100)


In [ ]:
# portflio return,portflio volatility,portflio sharpe ratio
print("Expected Average Return :",round(ret*100,2),"%")
print("Expected Volatility:", round(vol*100,2),"%")
print("Sharpe Ratio:",round(sr*100,2),"%")
print("Max Drawdown:",drawdown,"%")

In [ ]:
live_price = pd.read_csv("live trading.csv")["Price"].dropna()
live_daily_return = np.log(live_price/live_price.shift(1))
live_ret = live_daily_return.mean()
live_vol = live_daily_return.std()
live_sr = live_ret/vol

return_live = list(live_price)
mdd_live = MaxDrawdown(return_live)
drawdown_live = round(mdd_live*100)

In [ ]:
print("Expected Average Return :",round(live_ret*100,2),"%")
print("Expected Volatility:", round(live_vol*100,2),"%")
print("Sharpe Ratio:",round(live_sr*100,2),"%")
print("Max Drawdown:",drawdown_live,"%")